In [1]:
import $file.^.Magic

Compiling /workspace/Magic.sc


import $file.$

In [2]:
Magic.!("pip", "install", "torch==2.4.*")

In [3]:
import $ivy.`dev.scalapy::scalapy-core:0.5.3`

import $ivy.$

In [4]:
import me.shadaj.scalapy.py
import py.SeqConverters

val torch = py.module("torch")

val inputs = torch.tensor(
  Seq(
    Seq(0.43, 0.15, 0.89), // word1
    Seq(0.55, 0.87, 0.66), // word2
    Seq(0.57, 0.85, 0.64), // word3
    Seq(0.22, 0.58, 0.33), // word4
    Seq(0.77, 0.25, 0.10), // word5
    Seq(0.05, 0.80, 0.55)  // word6
  ).toPythonProxy
)

import me.shadaj.scalapy.py
import py.SeqConverters
torch: py.Module = <module 'torch' from '/usr/local/lib/python3.12/site-packages/torch/__init__.py'>
inputs: py.Dynamic = tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

In [5]:
import py.PyQuote

val attentionScores = py"$inputs @ $inputs.T"
println(attentionScores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


import py.PyQuote
attentionScores: py.Dynamic = tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [6]:
val attentionWeights = torch.softmax(attentionScores, dim = -1)
println(attentionWeights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


attentionWeights: py.Dynamic = tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [7]:
val contextVectors = py"$attentionWeights @ $inputs"
println(contextVectors)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


contextVectors: py.Dynamic = tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

In [32]:
// Workaround to define a class that inherits from a Python class
py.exec {
  s"""import torch.nn as nn
     |
     |class SelfAttention(nn.Module):
     |  def __init__(self, forward_method):
     |    super().__init__()
     |    self.forward_method = forward_method
     |  
     |  def forward(self, inputs):
     |    return self.forward_method(inputs)
     |""".stripMargin
}
type TorchTensor = py.Dynamic
def SelfAttentionV1(
  inputDimension: Int,
  outputDimension: Int
): py.Dynamic = {
  val weightsQuery = torch.nn.Parameter(torch.rand(inputDimension, outputDimension))
  val weightsKey = torch.nn.Parameter(torch.rand(inputDimension, outputDimension))
  val weightsValue = torch.nn.Parameter(torch.rand(inputDimension, outputDimension))
  val forwardMethod = (inputs: TorchTensor) => {
    val queries = py"$inputs @ $weightsQuery"
    val keys = py"$inputs @ $weightsKey"
    val values = py"$inputs @ $weightsValue"
    val attentionScores = py"$queries @ $keys.T"
    val attentionWeights = torch.softmax(py"$attentionScores / $outputDimension**0.5", dim = -1)
    py"$attentionWeights @ $values"
  }
  py.Dynamic.global.SelfAttention(forwardMethod)
}

defined type TorchTensor
defined function SelfAttentionV1

In [33]:
torch.manual_seed(123)
val selfAttentionV1 = SelfAttentionV1(inputDimension = inputs.shape.bracketAccess(1).as[Int], outputDimension = 2)
println(selfAttentionV1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


res33_0: py.Dynamic = <torch._C.Generator object at 0xfffeb3d798f0>
selfAttentionV1: py.Dynamic = SelfAttention()

In [34]:
def SelfAttentionV2(
  inputDimension: Int,
  outputDimension: Int
): py.Dynamic = {
  val weightsQuery = torch.nn.Linear(inputDimension, outputDimension, bias = false)
  val weightsKey = torch.nn.Linear(inputDimension, outputDimension, bias = false)
  val weightsValue = torch.nn.Linear(inputDimension, outputDimension, bias = false)
  val forwardMethod = (inputs: TorchTensor) => {
    val queries = weightsQuery(inputs)
    val keys = weightsKey(inputs)
    val values = weightsValue(inputs)
    val attentionScores = py"$queries @ $keys.T"
    val attentionWeights = torch.softmax(py"$attentionScores / $outputDimension**0.5", dim = -1)
    py"$attentionWeights @ $values"
  }
  py.Dynamic.global.SelfAttention(forwardMethod)
}

defined function SelfAttentionV2

In [35]:
torch.manual_seed(789)
val selfAttentionV2 = SelfAttentionV2(inputDimension = inputs.shape.bracketAccess(1).as[Int], outputDimension = 2)
println(selfAttentionV2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


res35_0: py.Dynamic = <torch._C.Generator object at 0xfffeb3d798f0>
selfAttentionV2: py.Dynamic = SelfAttention()

In [62]:
// Workaround to define a class that inherits from a Python class
py.exec {
  s"""import torch.nn as nn
     |
     |class CasualAttention(nn.Module):
     |  def __init__(self, init):
     |    super().__init__()
     |    init(self)
     |""".stripMargin
}
type Mask = py.Dynamic
def CasualAttention(
  inputDimension: Int,
  outputDimension: Int,
  dropoutProbability: Double,
  contextLength: Int
): py.Dynamic = {
  val weightsQuery = torch.nn.Linear(inputDimension, outputDimension, bias = false)
  val weightsKey = torch.nn.Linear(inputDimension, outputDimension, bias = false)
  val weightsValue = torch.nn.Linear(inputDimension, outputDimension, bias = false)
  val dropout = torch.nn.Dropout(dropoutProbability)

  val init = (self: py.Dynamic) => {
    self.register_buffer("mask", torch.triu(torch.ones(contextLength, contextLength), diagonal = 1))
    val mask = self.mask
      
    val forward = (batchedInputs: TorchTensor) => {
      val queries = weightsQuery(batchedInputs)
      val keys = weightsKey(batchedInputs)
      val values = weightsValue(batchedInputs)
      val attentionScores = py"$queries @ $keys.transpose(1, 2)"
      val tokensCount = batchedInputs.shape.bracketAccess(1).as[Int]
      attentionScores.masked_fill_(py"$mask.bool()[:$tokensCount, :$tokensCount]", -torch.inf)
      val attentionWeights = dropout(torch.softmax(py"$attentionScores / $outputDimension**0.5", dim = -1))
      py"$attentionWeights @ $values"
    }
    py.Dynamic.global.CasualAttention.forward = forward
  }
  py.Dynamic.global.CasualAttention(init)
}

defined type Mask
defined function CasualAttention

In [63]:
torch.manual_seed(123)
val batchedInputs = torch.stack((inputs, inputs), dim = 0)
println(s"batchedInputs.shape = ${batchedInputs.shape}")
val contextLength = batchedInputs.shape.bracketAccess(1).as[Int]
val casualAttention = CasualAttention(
  inputDimension = batchedInputs.shape.bracketAccess(2).as[Int], 
  outputDimension = 2, 
  dropoutProbability = 0.0, 
  contextLength
)
println(casualAttention(batchedInputs))

batchedInputs.shape = torch.Size([2, 6, 3])
tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)


res63_0: py.Dynamic = <torch._C.Generator object at 0xfffeb3d798f0>
batchedInputs: py.Dynamic = tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])
contextLength: Int = 6
casualAttention: py.Dynamic = CasualAttention()

In [64]:
// Workaround to define a class that inherits from a Python class
py.exec {
  s"""import torch.nn as nn
     |
     |class MultiHeadAttentionWrapper(nn.Module):
     |  def __init__(self):
     |    super().__init__()
     |""".stripMargin
}
def MultiHeadAttentionWrapper(
  inputDimension: Int,
  outputDimension: Int,
  dropoutProbability: Double,
  contextLength: Int,
  headsCount: Int
): py.Dynamic = {
  val heads = torch.nn.ModuleList(
    Seq.fill(headsCount)(
      CasualAttention(inputDimension, outputDimension, dropoutProbability, contextLength)
    ).toPythonProxy
  )
  val forward = (batchedInputs: TorchTensor) => {
    torch.cat(py"[head($batchedInputs) for head in $heads]", dim = -1)
  }
  py.Dynamic.global.MultiHeadAttentionWrapper.forward = forward
  py.Dynamic.global.MultiHeadAttentionWrapper()
}

defined function MultiHeadAttentionWrapper

In [71]:
torch.manual_seed(123)
val multiHeadAttentionWrapper = MultiHeadAttentionWrapper(
  inputDimension = batchedInputs.shape.bracketAccess(2).as[Int], 
  outputDimension = 2, 
  dropoutProbability = 0.0, 
  contextLength,
  headsCount = 2
)
println(multiHeadAttentionWrapper(batchedInputs))

tensor([[[0.4772, 0.1063, 0.4772, 0.1063],
         [0.5891, 0.3257, 0.5891, 0.3257],
         [0.6202, 0.3860, 0.6202, 0.3860],
         [0.5478, 0.3589, 0.5478, 0.3589],
         [0.5321, 0.3428, 0.5321, 0.3428],
         [0.5077, 0.3493, 0.5077, 0.3493]],

        [[0.4772, 0.1063, 0.4772, 0.1063],
         [0.5891, 0.3257, 0.5891, 0.3257],
         [0.6202, 0.3860, 0.6202, 0.3860],
         [0.5478, 0.3589, 0.5478, 0.3589],
         [0.5321, 0.3428, 0.5321, 0.3428],
         [0.5077, 0.3493, 0.5077, 0.3493]]], grad_fn=<CatBackward0>)


res71_0: py.Dynamic = <torch._C.Generator object at 0xfffeb3d798f0>
multiHeadAttentionWrapper: py.Dynamic = MultiHeadAttentionWrapper()

In [72]:
// Exercise 3.2
torch.manual_seed(123)
val multiHeadAttentionWrapper2 = MultiHeadAttentionWrapper(
  inputDimension = batchedInputs.shape.bracketAccess(2).as[Int], 
  outputDimension = 1, 
  dropoutProbability = 0.0, 
  contextLength,
  headsCount = 2
)
println(multiHeadAttentionWrapper2(batchedInputs))

tensor([[[ 0.2216,  0.2216],
         [ 0.0155,  0.0155],
         [-0.0546, -0.0546],
         [-0.0817, -0.0817],
         [-0.0957, -0.0957],
         [-0.1065, -0.1065]],

        [[ 0.2216,  0.2216],
         [ 0.0155,  0.0155],
         [-0.0546, -0.0546],
         [-0.0817, -0.0817],
         [-0.0957, -0.0957],
         [-0.1065, -0.1065]]], grad_fn=<CatBackward0>)


res72_0: py.Dynamic = <torch._C.Generator object at 0xfffeb3d798f0>
multiHeadAttentionWrapper2: py.Dynamic = MultiHeadAttentionWrapper()

In [73]:
// Workaround to define a class that inherits from a Python class
py.exec {
  s"""import torch.nn as nn
     |
     |class MultiHeadAttention(nn.Module):
     |  def __init__(self, init):
     |    super().__init__()
     |    init(self)
     |""".stripMargin
}
type Mask = py.Dynamic
def MultiHeadAttention(
  inputDimension: Int,
  outputDimension: Int,
  dropoutProbability: Double,
  contextLength: Int,
  headsCount: Int
): py.Dynamic = {
  assert(outputDimension % headsCount == 0, "Output dimension must be a multiple of heads count")
  val headDimension = outputDimension / headsCount
    
  val weightsQuery = torch.nn.Linear(inputDimension, outputDimension, bias = false)
  val weightsKey = torch.nn.Linear(inputDimension, outputDimension, bias = false)
  val weightsValue = torch.nn.Linear(inputDimension, outputDimension, bias = false)
  val outputProjection = torch.nn.Linear(outputDimension, outputDimension)
  val dropout = torch.nn.Dropout(dropoutProbability)
    
  val init = (self: py.Dynamic) => {
    self.register_buffer("mask", torch.triu(torch.ones(contextLength, contextLength), diagonal = 1))
    val mask = self.mask
      
    val forward = (batchedInputs: TorchTensor) => {
      val (batchesCount, tokensCount, tokenDimension) = batchedInputs.shape.as[(Int, Int, Int)]
      val queries = weightsQuery(batchedInputs)
        .view(batchesCount, tokensCount, headsCount, headDimension)
        .transpose(1, 2)
      val keys = weightsKey(batchedInputs)
        .view(batchesCount, tokensCount, headsCount, headDimension)
        .transpose(1, 2)
      val values = weightsValue(batchedInputs)
        .view(batchesCount, tokensCount, headsCount, headDimension)
        .transpose(1, 2)
      val attentionScores = py"$queries @ $keys.transpose(2, 3)"
      attentionScores.masked_fill_(py"$mask.bool()[:$tokensCount, :$tokensCount]", -torch.inf)
      val attentionWeights = dropout(torch.softmax(py"$attentionScores / $outputDimension**0.5", dim = -1))
      outputProjection(
        py"$attentionWeights @ $values"
          .transpose(1, 2)
          .contiguous()
          .view(batchesCount, tokensCount, outputDimension)
      )
    }
    py.Dynamic.global.CasualAttention.forward = forward
  }
  py.Dynamic.global.CasualAttention(init)
}

defined type Mask
defined function MultiHeadAttention

In [74]:
torch.manual_seed(123)
val multiHeadAttention = MultiHeadAttention(
  inputDimension = batchedInputs.shape.bracketAccess(2).as[Int], 
  outputDimension = 2, 
  dropoutProbability = 0.0,
  contextLength,
  headsCount = 2
)
println(multiHeadAttention(batchedInputs))

tensor([[[0.3190, 0.4858],
         [0.2941, 0.3906],
         [0.2854, 0.3600],
         [0.2692, 0.3879],
         [0.2636, 0.3935],
         [0.2574, 0.4033]],

        [[0.3190, 0.4858],
         [0.2941, 0.3906],
         [0.2854, 0.3600],
         [0.2692, 0.3879],
         [0.2636, 0.3935],
         [0.2574, 0.4033]]], grad_fn=<ViewBackward0>)


res74_0: py.Dynamic = <torch._C.Generator object at 0xfffeb3d798f0>
multiHeadAttention: py.Dynamic = CasualAttention()